This notebook does:
* Inspect the 5 dataset.
* Concatenate, clean data and feature engineering.
* Create **data quality reports with sweetviz**.
* Short summary of **findings about descriptive statistics**.
* Add a variable "domestic" from external data (then drop because all were domestic).
* Create the **data/cleaned_df.parquet**.

In [3]:
# Check whether all csv files contain the same variables.
import pandas as pd

# List of your CSV files (with folder path)
files = [f"data/raw/{year}_.csv" for year in range(2014, 2019)]

# Dictionary to hold column sets
columns_dict = {}

# Read each file and store its columns
for file in files:
    df = pd.read_csv(file, nrows=0)  # only read header
    columns_dict[file] = set(df.columns)

# Compare columns across files
all_equal = all(cols == list(columns_dict.values())[0] for cols in columns_dict.values())

if all_equal:
    print("✅ All files have the same variables (columns).")
else:
    print("❌ Files have different variables.")
    for file, cols in columns_dict.items():
        print(f"\n{file}:")
        print(sorted(cols))

    # Show differences between files
    common = set.intersection(*columns_dict.values())
    union = set.union(*columns_dict.values())
    print("\nVariables missing in some files:")
    print(sorted(union - common))


✅ All files have the same variables (columns).


In [4]:
# Load data: append the data files and show df

# Build list of file paths
files = [f"data/raw/{year}_.csv" for year in range(2014, 2019)]

# Load all files into a list of DataFrames with a year column
dfs = []
for file in files:
    year = os.path.basename(file).split("_")[0]   # extract '2014' from '2014_.csv'
    df = pd.read_csv(file)
    df["year"] = int(year)
    dfs.append(df)

# Concatenate all into one DataFrame
df = pd.concat(dfs, ignore_index=True)

# Show the first rows
df.head()

C:\Users\csoka\AppData\Local\Temp\ipykernel_1936\350552192.py:10: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


,Unnamed: 0,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,...,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed..27,year
0,11484,2014-01-01,AS,1,DCA,SEA,745,733.0,-12.0,8.0,...,349.0,332.0,2329,NaN,NaN,NaN,NaN,NaN,NaN,2014
1,11485,2014-01-01,AS,2,SEA,DCA,1410,1405.0,-5.0,13.0,...,278.0,261.0,2329,NaN,NaN,NaN,NaN,NaN,NaN,2014
2,11486,2014-01-01,AS,3,DCA,SEA,1840,1839.0,-1.0,15.0,...,339.0,320.0,2329,NaN,NaN,NaN,NaN,NaN,NaN,2014
3,11487,2014-01-01,AS,4,SEA,DCA,810,810.0,0.0,31.0,...,294.0,259.0,2329,NaN,NaN,NaN,NaN,NaN,NaN,2014
4,11488,2014-01-01,AS,5,EWR,SEA,1825,1829.0,4.0,20.0,...,371.0,347.0,2402,NaN,NaN,NaN,NaN,NaN,NaN,2014


In [5]:
# Check whether all op_carrier values are AS. If yes, then it contains no information, we can drop it
# True if every row has OP_CARRIER == "AS"
df["OP_CARRIER"].eq("AS").all()


True

In [6]:
# Check whether op_carrier_fl_num contains extra information (other than route: destination and origin)
df['flight_id'] = df['OP_CARRIER'].astype(str) + '_' + df['OP_CARRIER_FL_NUM'].astype(str)
df['route'] = df['ORIGIN'] + '-' + df['DEST']

cons = (df.groupby('flight_id')['route']
          .agg(n='size', n_mode=lambda s: s.value_counts().iloc[0],
               route_mode=lambda s: s.value_counts().idxmax()))
cons['consistency'] = cons['n_mode'] / cons['n']
print(cons['consistency'].describe())   # Calculates, per flight_id, the share of records on its dominant route.
# if ~>0.9, then flight_id tipically means the same route (in 90% it flies the same route, in 10% not)


count    1455.000000
mean        0.845410
std         0.207236
min         0.183071
25%         0.700000
50%         0.976641
75%         1.000000
max         1.000000
Name: consistency, dtype: float64


It suggests that op_carrier_fl_num contains limited extra signal (on top of destination and origin). So I drop it, but might worth reconsidering. For generalizable model one should drop it, for predicting within a small time frame if could be useful (might contain extra info on rotation, air craft type, other operational characteristics eg. gate, programs such as shuttle/seasonal/long/short)

In [7]:
# check variable types
df.dtypes

Unnamed: 0               int64
FL_DATE                 object
OP_CARRIER              object
OP_CARRIER_FL_NUM        int64
ORIGIN                  object
DEST                    object
CRS_DEP_TIME             int64
DEP_TIME               float64
DEP_DELAY              float64
TAXI_OUT               float64
WHEELS_OFF             float64
WHEELS_ON              float64
TAXI_IN                float64
CRS_ARR_TIME             int64
ARR_TIME               float64
ARR_DELAY              float64
CANCELLED              float64
CANCELLATION_CODE       object
DIVERTED               float64
CRS_ELAPSED_TIME         int64
ACTUAL_ELAPSED_TIME    float64
AIR_TIME               float64
DISTANCE                 int64
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
Unnamed..27            float64
year                     int64
flight_id               object
route                   object
dtype: o

In [8]:
# Keep only those that can be used for prediction, rename to snake_case and convert to proper varable type
# 1) Keep only the requested columns
keep = [
    "FL_DATE", "ORIGIN", "DEST",
    "CRS_DEP_TIME", "DEP_DELAY", "CRS_ARR_TIME", "CRS_ELAPSED_TIME",
    "DISTANCE", "year"
]

df = df[keep].copy()

# 2) Rename to snake_case
df = df.rename(columns={
    "FL_DATE": "fl_date",
    "ORIGIN": "origin",
    "DEST": "dest",
    "CRS_DEP_TIME": "crs_dep_time",
    "DEP_DELAY": "dep_delay",
    "CRS_ARR_TIME": "crs_arr_time",
    "CRS_ELAPSED_TIME": "crs_elapsed_time",
    "DISTANCE": "distance",
})

# --- helpers ---
def hhmm_to_time(series):
    """
    Convert hhmm integers (e.g., 745 -> 07:45) to datetime.time.
    Handles floats/strings/NaN safely. Treats 2400 as 0000.
    """
    s = pd.to_numeric(series, errors="coerce").round().astype("Int64")
    s = s.where(s != 2400, 0)  # 2400 -> 0000
    t = pd.to_datetime(s.astype(str).str.zfill(4), format="%H%M", errors="coerce")
    return t.dt.time  # object dtype of datetime.time

# 3) Type conversions
# fl_date → date
df["fl_date"] = pd.to_datetime(df["fl_date"], errors="coerce").dt.date

# origin, dest → category
df["origin"] = df["origin"].astype("category")
df["dest"]   = df["dest"].astype("category")

# CRS times → time/duration
df["crs_dep_time"] = hhmm_to_time(df["crs_dep_time"])
df["crs_arr_time"] = hhmm_to_time(df["crs_arr_time"])

df.head()


,fl_date,origin,dest,crs_dep_time,dep_delay,crs_arr_time,crs_elapsed_time,distance,year
0,2014-01-01,DCA,SEA,07:45:00,-12.0,10:37:00,352,2329,2014
1,2014-01-01,SEA,DCA,14:10:00,-5.0,21:56:00,286,2329,2014
2,2014-01-01,DCA,SEA,18:40:00,-1.0,21:45:00,365,2329,2014
3,2014-01-01,SEA,DCA,08:10:00,0.0,16:06:00,296,2329,2014
4,2014-01-01,EWR,SEA,18:25:00,4.0,21:52:00,387,2402,2014


In [9]:
df.dtypes

fl_date               object
origin              category
dest                category
crs_dep_time          object
dep_delay            float64
crs_arr_time          object
crs_elapsed_time       int64
distance               int64
year                   int64
dtype: object

Feature engineering: 
1. route (from origin and dest) 
2. proxy for holidays, peak season
3. weekday, weekend
4. hour, month, season
5. distance bucket
6. bank pressure (origin departures per hour)

In [10]:
# Safe datetime view of fl_date (doesn't overwrite your column)
dt = pd.to_datetime(df["fl_date"], errors="coerce")

def extract_hour_from_time(col):
    """Gets hour from either timedelta64 or Python datetime.time (object)."""
    if pd.api.types.is_timedelta64_dtype(df[col]):
        return df[col].dt.components.hours
    return df[col].apply(lambda t: t.hour if pd.notnull(t) else np.nan)

# ----- requested features -----

# 1) Route
df["route"] = df["origin"].astype(str) + "-" + df["dest"].astype(str)

# 2) Holiday / peak season proxies (US-centric)
cal = USFederalHolidayCalendar()
if dt.notna().any():
    holidays = cal.holidays(start=dt.min(), end=dt.max())
    df["is_us_holiday"] = dt.isin(holidays).fillna(False)

    if len(holidays):
        near = pd.DatetimeIndex(holidays).union(holidays - pd.Timedelta(days=1)).union(holidays + pd.Timedelta(days=1))
        df["near_holiday"] = dt.isin(near).fillna(False)
    else:
        df["near_holiday"] = False
else:
    df["is_us_holiday"] = False
    df["near_holiday"] = False

df["is_summer_peak"] = dt.dt.month.isin([6, 7, 8]).fillna(False)

def thanksgiving_date(y):
    first = pd.Timestamp(y, 11, 1)
    first_thu = first + pd.Timedelta(days=(3 - first.weekday()) % 7)
    return first_thu + pd.Timedelta(days=21)

is_tg = pd.Series(False, index=df.index)
for y in pd.Index(dt.dt.year.dropna().astype(int).unique()):
    tg = thanksgiving_date(int(y))
    is_tg |= (dt >= tg - pd.Timedelta(days=3)) & (dt <= tg + pd.Timedelta(days=3))
df["is_thanksgiving_week"] = is_tg

df["is_xmas_newyear"] = ((dt.dt.month == 12) & (dt.dt.day >= 20)) | ((dt.dt.month == 1) & (dt.dt.day <= 6))
df["is_xmas_newyear"] = df["is_xmas_newyear"].fillna(False)

df["is_peak_season"] = df[["is_summer_peak", "is_thanksgiving_week", "is_xmas_newyear", "near_holiday"]].any(axis=1)

# 3) Weekday / Weekend
df["weekday"] = dt.dt.day_name()
df["is_weekend"] = (dt.dt.weekday >= 5).fillna(False)

# 4) Time-related

# Departure hour (0–23) from scheduled dep time
df["dep_hour"] = extract_hour_from_time("crs_dep_time")

# Month / season / year
df["month"] = dt.dt.month
df["season"] = pd.Categorical(
    np.select(
        [
            df["month"].isin([12, 1, 2]),
            df["month"].isin([3, 4, 5]),
            df["month"].isin([6, 7, 8]),
            df["month"].isin([9, 10, 11]),
        ],
        ["winter", "spring", "summer", "autumn"],
        default="unknown",
    ),
    ordered=True, categories=["winter", "spring", "summer", "autumn"]
)

# Convert to categorical
df["year"] = df["year"].astype("category")
df["dep_hour"] = df["dep_hour"].astype("category")
df["month"] = df["month"].astype("category")

# 5) Distance buckets
df["distance_bin"] = pd.cut(pd.to_numeric(df["distance"], errors="coerce"),
                            bins=[-np.inf, 300, 1000, 2000, np.inf],
                            labels=["short", "medium", "long", "ultra"],
                            ordered=True)

# 6) Bank pressure (origin departures per hour)
# Count scheduled departures per (origin, date, dep_hour)
df["_dep_date"] = dt.dt.date
df["_dep_hour"] = df["dep_hour"].astype("Int64")   # may be NA if time missing

df["bank_pressure_hour"] = (
    df.groupby(["origin", "_dep_date", "_dep_hour"])["origin"]
      .transform("size")
      .astype("Int64")
)

# optional: fill missing with 0 if you prefer numeric
# df["bank_pressure_hour"] = df["bank_pressure_hour"].fillna(0).astype("Int64")

# clean up helper columns
df.drop(columns=["_dep_date", "_dep_hour"], inplace=True)


C:\Users\csoka\AppData\Local\Temp\ipykernel_1936\507700364.py:90: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["origin", "_dep_date", "_dep_hour"])["origin"]


In [11]:
df.head()

,fl_date,origin,dest,crs_dep_time,dep_delay,crs_arr_time,crs_elapsed_time,distance,year,route,...,is_thanksgiving_week,is_xmas_newyear,is_peak_season,weekday,is_weekend,dep_hour,month,season,distance_bin,bank_pressure_hour
0,2014-01-01,DCA,SEA,07:45:00,-12.0,10:37:00,352,2329,2014,DCA-SEA,...,False,True,True,Wednesday,False,7,1,winter,ultra,1
1,2014-01-01,SEA,DCA,14:10:00,-5.0,21:56:00,286,2329,2014,SEA-DCA,...,False,True,True,Wednesday,False,14,1,winter,ultra,9
2,2014-01-01,DCA,SEA,18:40:00,-1.0,21:45:00,365,2329,2014,DCA-SEA,...,False,True,True,Wednesday,False,18,1,winter,ultra,1
3,2014-01-01,SEA,DCA,08:10:00,0.0,16:06:00,296,2329,2014,SEA-DCA,...,False,True,True,Wednesday,False,8,1,winter,ultra,11
4,2014-01-01,EWR,SEA,18:25:00,4.0,21:52:00,387,2402,2014,EWR-SEA,...,False,True,True,Wednesday,False,18,1,winter,ultra,1


In [12]:
df.dtypes

fl_date                   object
origin                  category
dest                    category
crs_dep_time              object
dep_delay                float64
crs_arr_time              object
crs_elapsed_time           int64
distance                   int64
year                    category
route                     object
is_us_holiday               bool
near_holiday                bool
is_summer_peak              bool
is_thanksgiving_week        bool
is_xmas_newyear             bool
is_peak_season              bool
weekday                   object
is_weekend                  bool
dep_hour                category
month                   category
season                  category
distance_bin            category
bank_pressure_hour         Int64
dtype: object

In [13]:
# check duplicates
df.duplicated().sum()

0

In [14]:
# Generate and save Sweetviz data quality/EDA reports without opening

# Parse dates just for splitting (df['fl_date'] can stay object)
dt = pd.to_datetime(df["fl_date"], errors="coerce")

# Define cutoffs
cutoff_past  = pd.Timestamp("2018-05-31")
cutoff_predict = pd.Timestamp("2018-06-01")

# Create subsets
df_past   = df.loc[dt <= cutoff_past].copy()
df_predict = df.loc[dt >= cutoff_predict].copy()

print(f"Rows in PAST (<= 2018-05-31):  {len(df_past)}")
print(f"Rows in FUTURE (>= 2018-06-01): {len(df_predict)}")

sv.analyze(df_past).show_html("output/past_data_report.html", open_browser=False)
sv.analyze(df_predict).show_html("output/predict_data_report.html", open_browser=False)


Rows in PAST (<= 2018-05-31):  785392
Rows in FUTURE (>= 2018-06-01): 155495


Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


Report output/past_data_report.html was generated.


Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)

Report output/predict_data_report.html was generated.


Findings in the data quality report:
- **SEA is by far the most frequent** origin and destination.
- **Mean delay is 1 minute, median is -4**. Average and median **distance is around 1000** (km or mile?).
- Years show an increasing frequency, which means that (suppose that our sample is a 100% sample) there are more domestic flights operated by Alaska Airlines every year between 2014-2017 so we can expect an expansion.
- I **don't see any obvious signs of data problems**, no duplicates, no missing values (only in delay, but <1%), distribution of variables look ok.
- Although, there are **outliers in delay time**, we should think about how to clean it.

I also checked in the command line whether append was successful. To calculates the line count in all csv files in the data/raw/ folder, and adds these numbers: 
```bash
sum=0; for f in *.csv; do c=$(wc -l < "$f"); echo "$c  $f"; sum=$((sum+c)); done; echo "TOTAL $sum"



Extend the features with an external source: us the list of US airports to generate a dummy for domestic flights.
- origin_domestic is 1 if origin is in the list of airports
- dest_domestic is 1 if dest is in the list of airports

In [18]:
# Load domestic airport list
airports = pd.read_csv("data/raw/airports.csv")

# Normalize column names and build IATA set
airports.columns = airports.columns.str.strip().str.upper()
iata_set = set(
    airports["IATA"].dropna().astype(str).str.strip().str.upper().unique()
)

# Add dummies to your flights df
df["origin_domestic"] = (
    df["origin"].astype(str).str.strip().str.upper().isin(iata_set).astype("int8")
)
df["dest_domestic"] = (
    df["dest"].astype(str).str.strip().str.upper().isin(iata_set).astype("int8")
)

# quick check
print(df[["origin","dest","origin_domestic","dest_domestic"]].head())

  origin dest  origin_domestic  dest_domestic
0    DCA  SEA                1              1
1    SEA  DCA                1              1
2    DCA  SEA                1              1
3    SEA  DCA                1              1
4    EWR  SEA                1              1


In [21]:
print(df[["origin_domestic","dest_domestic"]].describe().T)

                    count  mean  std  min  25%  50%  75%  max
origin_domestic  940887.0   1.0  0.0  1.0  1.0  1.0  1.0  1.0
dest_domestic    940887.0   1.0  0.0  1.0  1.0  1.0  1.0  1.0


It seems that all flights are domestic, so we can drop this variable.

In [22]:
df = df.drop(columns=["origin_domestic", "dest_domestic"])

In [23]:
df.to_parquet("data/cleaned_df.parquet")  # fast + preserves dtypes